In [200]:
import pandas as pd
import numpy as np

In [201]:
name = 'GCI'
dataset = 'small' # small or big
train_val = '43' # 43 or 73

In [202]:
pf=pd.read_csv(f'datasets/{dataset}_matrix.csv')

In [203]:
# 用户、视频、播放率
user_id=np.array(pf['user_id'])
video_id=np.array(pf['video_id'])
watch_ratio=np.array(pf['watch_ratio'])

In [204]:
pc=pd.read_csv('datasets/item_categories.csv')

In [205]:
video_id_ca=np.array(pc['video_id'])

In [206]:
full_video_tag=[]
bad_video=[]
for i in range(pc['feat'].shape[0]):
    l=np.array(pc['feat'])[i][1:-1] # 去掉首尾的[]
    temp=[int(s) for s in l.split(',')] # 分割字符串，转化为int
    full_video_tag.append(temp)
    for j in range(len(temp)):
        if temp[j]==0:
            bad_video.append(i)

In [207]:
# 这3个视频没有标签
for i in range(len(bad_video)):
    if(len(full_video_tag[bad_video[i]])==1):
        print(video_id_ca[bad_video[i]])

2585
4255
7072


In [208]:
# 1411条交互记录是看过上面3条视频
dele=[]
for i in range(user_id.shape[0]):   # 4,676,570条交互记录
    if video_id[i]==2585 or video_id[i]==4255 or video_id[i]==7072:
        dele.append(i)

In [209]:
len(dele)

2614

In [210]:
user_id_new=[]
video_id_new=[]
watch_ratio_new=[]
k=0
# 删除掉上面1411条交互记录
for i in range(user_id.shape[0]):
    if dele[k]==i:
        if k<(len(dele)-1):
            k+=1
        continue
    else:
        user_id_new.append(user_id[i])
        video_id_new.append(video_id[i])
        watch_ratio_new.append(watch_ratio[i])

In [211]:
user_id_new=np.array(user_id_new)
video_id_new=np.array(video_id_new)
watch_ratio_new=np.array(watch_ratio_new)

In [212]:
new_watch_ratio=watch_ratio_new

In [213]:
len(set(user_id_new))   # 1411个用户

7176

In [214]:
len(set(video_id_new))  # 3326个视频

10725

In [215]:
dit_user=dict()
reve_user=dict()
dit_video=dict()
reve_video=dict()
ab_set_user=set(user_id_new)    # 1411个用户
ab_set_video=set(video_id_new)  # 3326个视频
k=0
for item in ab_set_user:    # 给用户编号，0-1410
    dit_user[k]=item        # {k: user_id}
    reve_user[item]=k       # {user_id: k}
    k+=1
k=0
for item in ab_set_video:   # 给视频编号，0-3325
    dit_video[k]=item       # {k: video_id}
    reve_video[item]=k      # {video_id: k}
    k+=1

In [216]:
new_user_id=[]
new_video_id=[]
for i in range(user_id_new.shape[0]):   # 4,675,159条交互记录-1411条交互记录
    new_user_id.append(reve_user[user_id_new[i]])   # 新的用户编号，0-1410
    new_video_id.append(reve_video[video_id_new[i]])    # 新的视频编号，0-3325

In [217]:
new_user_id=np.array(new_user_id)   # 4,675,159条交互记录-1411条交互记录
new_video_id=np.array(new_video_id)

In [218]:
shape_1=len(set(new_user_id))   # 1411个用户
shape_2=len(set(new_video_id))  # 3326个视频

In [219]:
video_list=[]
for i in range(shape_1):    # 1411个用户
    video_list.append([])

In [220]:
for i in range(new_user_id.shape[0]):   # 4M条交互记录
    if new_watch_ratio[i]>=2.0:
        video_list[new_user_id[i]].append(new_video_id[i])  #  同一个用户看过的所有视频

In [221]:
test_index=np.load(f'npy_for_pre_recall_f1/{name}/{dataset}idx_test{train_val}.npy')

In [222]:
test_index.shape

(7772,)

In [223]:
# train_index=np.load('npy_for_pre_recall_f1/ours/small_matrixidx_train43.npy')

In [224]:
# val_index=np.load('npy_for_pre_recall_f1/ours/small_matrixidx_val43.npy')

In [225]:
pred=np.load(f'npy_for_pre_recall_f1/{name}/{dataset}pred{train_val}.npy')

In [226]:
trued=np.load(f'npy_for_pre_recall_f1/{name}/{dataset}y_true{train_val}.npy')

In [227]:
import os
import numpy as np
import scipy.sparse as sp
import torch
import random
import torch.nn.functional as F
from collections import defaultdict

In [228]:
def get_acc(pred, label, c_train, c_val, model):
    # assume the c_train, c_val, c_test are ranked according to their c_ids
    mypred = torch.ones(pred.shape)*float('-inf')
    if(model == 'train'):
        mypred[:, :c_train] = pred[:, :c_train]
    elif model == 'val':
        mypred[:, c_train: c_train+c_val] = pred[:, c_train: c_train+c_val]
    elif model == 'test':
        mypred[:, c_train+c_val: ] = pred[:, c_train+c_val: ]
    return get_acc_basic(mypred, label,model) 

def get_acc_basic(predict, label,model):
    predict = torch.argmax(predict, axis=1)
    if model=='test':
        print(predict)
        print(label)
    acc = (label==predict)
    result = acc.sum().numpy()
    #print('Train true/false acc:', result/len(acc))
    return result/len(acc),predict

In [229]:
test_acc ,pre= get_acc(torch.tensor(pred[test_index]), torch.tensor(trued[test_index]), c_train=4, c_val=3, model='test')

tensor([ 7,  7,  7,  ..., 14, 10, 10])
tensor([ 7,  7,  7,  ..., 28, 28, 28])


In [230]:
set(np.array(pre))

{7, 10, 14, 19}

In [231]:
test_acc

0.2552753474009264

In [232]:
final_pred=trued.copy()

In [233]:
final_pred[test_index]=pre

In [234]:
len(video_list)

7176

In [235]:
from sklearn.metrics import precision_recall_fscore_support

In [236]:
import warnings
warnings.filterwarnings("ignore")

In [237]:
Precision=0
Recall=0
macroF1=0
k=0
for i in range(len(video_list)):
    if len(video_list[i])==0:
        k+=1
        continue
    temp_t=trued[video_list[i]]
    temp_p=final_pred[video_list[i]]
    temp_Precision,temp_Recall,temp_macroF1,_=precision_recall_fscore_support(temp_t, temp_p,average='macro')
    Precision+=temp_Precision
    Recall+=temp_Recall
    macroF1+=temp_macroF1

In [238]:
Precision/(len(video_list)-k)

0.3117480576469327

In [239]:
Recall/(len(video_list)-k)

0.3556145791827157

In [240]:
macroF1/(len(video_list)-k)

0.32101658998953